In [60]:
def clean(df):
    df = df.dropna(how='all')
    df.loc[df.Destination=='Details','Destination'] = np.nan
    df["Vendor"] = df["Vendor"].fillna(1)
    df = df.fillna(method="ffill")
    df = df[df.Vendor!=1]
    df.loc[df["Destination"] == "Loading...","Destination"] = np.nan
    df = df.fillna(method="ffill")
    df = df.drop('#',axis=1)
    #df.Prefix = df.Prefix.astype(int)
    return df

In [2]:
import numpy as np 

In [3]:
import requests
import pandas as pd

In [4]:
from bs4 import BeautifulSoup

In [5]:
url = "http://tool.ariatelekom.com/billing/index.php?/login"

In [44]:
s = requests.Session()

In [45]:
r= s.get(url)

In [46]:
r = s.post("http://tool.ariatelekom.com/billing/index.php?/login/login_do",data={"username":"umut","password":"d8a9123w"})

In [47]:
r.ok

True

In [48]:
soup = BeautifulSoup(r.content)

In [49]:
report_url = "http://tool.ariatelekom.com/billing/index.php?/reports/call_analysis"

In [50]:
start_date= end_date=None

In [51]:
params = {"i_vendor":"",
"i_connection": "",
"start_date_time_utc": "",
"end_date_time_utc": "",
"now_date_time_utc": "",
"hidden_by_connection": "1",
"hidden_i_routing": "",
"hidden_action": "",
"description": "",
"target": "",
"interval": "6",
"start_date": start_date,
"hour": "00",
"minute": "00",
"second": "00",
"start_time":"00:00:00",
"end_date": end_date,
"hour": "23",
"minute": "59",
"second": "59",
"end_time": "23:59:59",
"i_routing": "1",
"order": "2",
"by_connection": "1",
"action": "Update Report"}

In [52]:
dates = pd.date_range(start=pd.datetime(2018,8,1),end=pd.datetime(2018,9,12))

In [53]:
def get_by_date(date):
    params["start_date"] = params["end_date"] = date;
    r = s.post(report_url,data = params)
    return r

In [54]:
res = get_by_date("2018-08-01")

In [55]:
dates[::-1][0]

Timestamp('2018-09-12 00:00:00', freq='-1D')

In [56]:
res.reason

'OK'

In [57]:
soupres =  BeautifulSoup(res.content)

In [42]:
soupres.find("h1")

In [58]:
df = pd.read_html(str(soupres.find_all('table',{'id':'result'})[0]))

In [61]:
df_eski = clean(df[0])

In [33]:
df = clean(df[0])

In [65]:
df_eski["Destination"].unique().shape

(715,)

In [73]:
df_eski["Vendor"].unique().shape

(105,)

In [73]:
df.loc[df["Destination"] == "Loading...","Destination"] = np.nan

In [78]:
df = df.fillna(method="ffill")

In [357]:
df = df.drop('#',axis=1)

In [361]:

df.head()

,Destination,Prefix,Vendor,Target,TC,SC,Min,ACD,ASR,Blended
1,Afghanistan,93.0,VENTATELE PRE 32,5,29.0,2.0,9.09,04:34,689.0,0.1600
3,Afghanistan,93.0,TATA 26,5,21.0,0.0,0.00,0,0.0,0.1747
5,Afghanistan,93.0,LENSOL PRE 39,5,17.0,2.0,1.22,00:41,1170.0,0.1754
7,Afghanistan,93.0,DAX 528,5,12.0,0.0,0.00,0,0.0,0.0750
9,Afghanistan,93.0,MEA FS 243,5,12.0,0.0,0.00,0,0.0,0.1190


In [81]:
import re

In [309]:
def sippy_regex(rule,number):
    rule = rule.rstrip('/')
    rule = rule.replace('s/','')
    rulesp = rule.split('/')
    if len(rulesp) >= 2:
        match_s,replace_s = rulesp
    else:
        match_s = rulesp[0]
        replace_s=''
    pattern_match = re.compile(match_s)
    m = pattern_match.match(number)
    if m:
        number = pattern_match.sub(replace_s,number)
    else:
        print('No match')
    return text


In [ ]:
re.compile('')

In [283]:
sippy_regex("s/.*/${R:[0-3,6]5}/","0111245")

'${R:[0-3,6]5}'

In [221]:
text = "0112345"

In [209]:
sippy_regex("s/^011/567/","0112345")

('^011', '')

In [290]:
re.compile("^101", re.M).search("1012465", 0)

<_sre.SRE_Match object; span=(0, 3), match='101'>

In [297]:
pattern_match = re.compile('.*')

In [ ]:
re.sub()

In [304]:
re.sub(r'${R:[0-3,6]5',"asd")

TypeError: sub() missing 1 required positional argument: 'string'

In [277]:
m.group(1)

IndexError: no such group

In [275]:
text.replace('','#504')

'#5040#5041#5041#5042#5043#5044#5045#504'

In [198]:
if m:
    to_rep = m.group()
    text = text.replace(to_rep,replace_s)
else:
    print('No match')

In [199]:
text

'2345'

In [86]:
pattern = re.compile("^011")

In [87]:
pattern.match("01123")

<_sre.SRE_Match object; span=(0, 3), match='011'>

In [327]:
s = "s/^101//"

In [328]:
s = s.rstrip('/')